In [1]:
import transformers
from transformers import pipeline
import os

pipe = pipeline(task='text-classification', model='ProsusAI/finbert')

Device set to use cpu


In [2]:
total_score = 0
num_articles = 0

In [3]:
import pandas as pd

pwd = os.getcwd()

file_path = f'{pwd}/df.csv'
df = pd.read_csv(fr'{file_path}')

df.head()

,headline,date,description
0,Atlas Insurance and SOS Malta launch VR educat...,2025-03-04 12:26:00,Atlas Insurance and SOS Malta inaugurated thei...
1,MDB hosts 20 European institutions to discuss ...,2025-03-04 12:21:00,"The Malta Development Bank (MDB), in collabora..."
2,BOV encourages smarter money management with c...,2025-03-04 11:29:00,Bank of Valletta is launching a Cash Migration...
3,Maltese Payroll Software Company Buddy kicks o...,2025-03-04 09:21:00,Maltese technology leader Buddy is proud to an...
4,HSBC Malta Foundation supports K-9 Urban Searc...,2025-03-04 09:16:00,HSBC Malta Foundation is proud to support K-9 ...


In [4]:
sentiment_label = []

for index, row in df.iterrows():
    print(f'Headline: {row["headline"]}')
    print(f'Date: {row["date"]}')
    print(f'Description: {row["description"]}')
    
    sentiment = pipe(row['description'])[0]
    print(f'Sentiment: {sentiment}')
    
    print('-' * 40)
    if sentiment['label'] == 'positive':
        total_score += sentiment['score']
        num_articles += 1
    elif sentiment['label'] == 'negative':
        total_score -= sentiment['score']
        num_articles += 1
        
    sentiment_label.append(sentiment['label'])
    df.at[index, 'sentiment'] = sentiment['label']
    df.at[index, 'sentiment_score'] = sentiment['score']
    
df.to_csv(fr'{file_path}', index=False)

final_score = total_score / num_articles if num_articles > 0 else 0


Headline: Atlas Insurance and SOS Malta launch VR education initiative in Maltese schools
Date: 2025-03-04 12:26:00
Description: Atlas Insurance and SOS Malta inaugurated their three-year initiative to integrate virtual reality (VR) technology into the curricula of two independent schools and one church school. This project...
Sentiment: {'label': 'positive', 'score': 0.7671111822128296}
----------------------------------------
Headline: MDB hosts 20 European institutions to discuss the challenges of digital financing
Date: 2025-03-04 12:21:00
Description: The Malta Development Bank (MDB), in collaboration with the European Association of Guarantee Institutions (AECM), hosted a high-level technical workshop on digital financing, bringing together 20...
Sentiment: {'label': 'positive', 'score': 0.7015870213508606}
----------------------------------------
Headline: BOV encourages smarter money management with cash migration incentive
Date: 2025-03-04 11:29:00
Description: Bank of Vallett

In [5]:
# After your existing code, add:

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])
# Extract just the date part (removing time)
df['date'] = df['date'].dt.date

# Group by date and get majority sentiment
daily_sentiment = df.groupby('date')['sentiment'].agg(lambda x: x.value_counts().index[0]).reset_index()
# Get sentiment counts per day
daily_sentiment_counts = df.groupby(['date', 'sentiment']).size().unstack(fill_value=0)

# Print results
for date in daily_sentiment['date']:
    print(f'\nDate: {date}')
    print('Sentiment counts:')
    print(daily_sentiment_counts.loc[date])
    print(f"Majority sentiment: {daily_sentiment[daily_sentiment['date'] == date]['sentiment'].values[0]}")


Date: 2019-02-26
Sentiment counts:
sentiment
negative    0
neutral     1
positive    0
Name: 2019-02-26, dtype: int64
Majority sentiment: neutral

Date: 2019-02-27
Sentiment counts:
sentiment
negative    0
neutral     0
positive    3
Name: 2019-02-27, dtype: int64
Majority sentiment: positive

Date: 2019-02-28
Sentiment counts:
sentiment
negative    0
neutral     0
positive    1
Name: 2019-02-28, dtype: int64
Majority sentiment: positive

Date: 2019-03-01
Sentiment counts:
sentiment
negative    1
neutral     0
positive    0
Name: 2019-03-01, dtype: int64
Majority sentiment: negative

Date: 2019-03-04
Sentiment counts:
sentiment
negative    0
neutral     0
positive    1
Name: 2019-03-04, dtype: int64
Majority sentiment: positive

Date: 2019-03-05
Sentiment counts:
sentiment
negative    1
neutral     0
positive    1
Name: 2019-03-05, dtype: int64
Majority sentiment: negative

Date: 2019-03-06
Sentiment counts:
sentiment
negative    1
neutral     0
positive    1
Name: 2019-03-06, dtype: 

In [ ]:
# Group by date and get majority sentiment
majority_sentiment = df.groupby('date')['sentiment'].agg(lambda x: x.value_counts().index[0]).reset_index()
majority_sentiment.columns = ['date', 'majority_sentiment']

# Calculate average sentiment for each date
average_sentiment = df.groupby('date')['sentiment_score'].mean().reset_index()
average_sentiment.columns = ['date', 'average_sentiment']

# Merge the results
aggregated_sentiment = pd.merge(majority_sentiment, average_sentiment, on='date')

# Display the aggregated sentiment
print(aggregated_sentiment.head())

         date majority_sentiment  average_sentiment
0  2019-02-26            neutral           0.824267
1  2019-02-27           positive           0.882053
2  2019-02-28           positive           0.947680
3  2019-03-01           negative           0.972589
4  2019-03-04           positive           0.936579


In [ ]:
# aggregate_sentiment to csv
aggregated_sentiment.to_csv(fr'{pwd}/aggregated_sentiment.csv', index=False)

In [20]:
aggregated_sentiment = pd.read_csv(fr'{pwd}/aggregated_sentiment.csv')
aggregated_sentiment.head()

,date,majority_sentiment,average_sentiment
0,2019-02-26,0,0.824267
1,2019-02-27,2,0.882053
2,2019-02-28,2,0.947680
3,2019-03-01,1,0.972589
4,2019-03-04,2,0.936579


In [21]:
stock_df = pd.read_csv(fr'{pwd}/stock_df.csv')
stock_df.head()

,Date,Open,Close,Change
0,2025-03-03,3915.44,3917.741,0.058793
1,2025-02-28,3913.54,3915.439,0.048601
2,2025-02-27,3908.73,3913.537,0.122904
3,2025-02-26,3910.01,3908.733,-0.032736
4,2025-02-25,3920.37,3910.013,-0.264057
